# 📘 Task Explanation: CUDA Matrix Multiplication Optimization  
*(Naive → Block Tiling → Shared Memory)*

This task guides you through a **three-day progressive implementation and optimization** of CUDA matrix multiplication (GEMM).  
You will start from a **naive implementation**, then introduce **block tiling**, and finally optimize using **shared memory with bank-conflict awareness**.  
The goal is to build a complete and systematic understanding of GPU optimization.

---

## 🗓️ Day 1 — Implement Naive MatMul Kernel & Test Correctness

### 🎯 Objective
Implement the most straightforward CUDA matrix multiplication kernel to serve as a **performance baseline** for later optimizations, and ensure the result is correct.

### 🧩 Tasks
- Write a naive CUDA kernel:
  - Each thread computes one element of matrix `C`
  - All operands are read directly from **global memory**
- Implement a CPU reference version
- Compare GPU results with CPU results to verify correctness

### 🧠 Key Concepts
- Mapping 2D thread indices to matrix row/column indices
- Understanding why the naive implementation repeatedly loads elements from `A` and `B`
- Recognizing why this version is typically **memory-bound**

### 📦 Deliverables
- Naive CUDA MatMul kernel
- Correctness test (PASS / FAIL)
- Baseline performance measurement

---

## 🗓️ Day 2 — Implement Block Tiling & Configure Tile Size

### 🎯 Objective
Introduce **block-level tiling** so that threads within a block cooperate on computing a submatrix, laying the structural foundation for shared-memory optimization.

### 🧩 Tasks
- Decompose matrices into fixed-size tiles (e.g., 16×16, 32×32)
- Assign each thread block to compute one tile of matrix `C`
- Use 2D block and thread indexing to map elements within a tile
- Experiment with different tile sizes (16×16 vs 32×32)

### 🧠 Key Concepts
- How tiling improves data locality
- How tile size affects:
  - Parallelism
  - Occupancy
  - Resource usage
- Why block tiling alone does **not yet reduce global memory traffic**

### 📦 Deliverables
- Block-tiled MatMul kernel
- Performance comparison across different tile sizes

---

## 🗓️ Day 3 — Cache Tiles in Shared Memory & Handle Bank Conflicts

### 🎯 Objective
Use **shared memory** to cache matrix tiles, reduce global memory accesses, and analyze and mitigate **shared-memory bank conflicts**.

### 🧩 Tasks
- Load tiles of matrices `A` and `B` into shared memory
- Synchronize threads within a block using `__syncthreads()`
- Perform multiply–accumulate operations using shared memory
- Analyze shared-memory access patterns
- Identify and mitigate potential bank conflicts (e.g., via padding)

### 🧠 Key Concepts
- How shared memory enables data reuse and reduces global memory traffic
- How bank conflicts serialize shared-memory accesses and hurt performance
- Why certain tile layouts naturally avoid bank conflicts
- The role of padding in shared-memory layouts

### 📦 Deliverables
- Shared-memory tiled MatMul kernel
- Bank-conflict analysis and optimization notes
- Performance comparison against naive and block-tiling versions

---

## 🎓 What You Should Learn from This Task
By completing this task, you should understand:
- The full optimization pipeline for CUDA GEMM
- Why performance improves when moving from **global memory → shared memory**
- Trade-offs between tile size and hardware resources
- The real impact of bank conflicts on performance

---

## 🚀 Relevance to ML Systems
Matrix multiplication is the computational core of:
- cuBLAS GEMM
- Linear and Attention layers in Transformers
- Triton matmul kernels
- Tensor Core–based acceleration

Completing this task means you have built **GPU kernel engineer–level fundamentals** for ML systems and high-performance computing.

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Sun Jan 11 12:14:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8       

In [ ]:
!apt-get update
!apt-get install -y cuda-toolkit-12-4

In [21]:
%%writefile matmul_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>


#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

// -----------------------------
// TODO: set TILE size (try 16 and 32)
// -----------------------------
#ifndef TILE
#define TILE 16  // TODO
#endif

// ============================================================
// Day 1: Naive MatMul Kernel (Baseline)
// C = A * B
// A: MxK, B: KxN, C: MxN (row-major)
// ============================================================
__global__ void matmulNaive(const float* __restrict__ A,
                            const float* __restrict__ B,
                            float* __restrict__ C,
                            int M, int N, int K) {
    // TODO:
    // - compute (row, col) from blockIdx/threadIdx
    // - guard: if row < M && col < N
    // - compute dot product over k in [0, K)
    // - write C[row*N + col]

    int row = blockIdx.y * TILE + threadIdx.y;
    int col = blockIdx.x * TILE + threadIdx.x;

    float acc = 0.0f;
    if (row < M && col < N){
        for (int k = 0; k < K; ++k){
            acc += A[row * K + k] * B[k*N + col];
        }
        C[row * N + col] = acc;
    }
}


// ============================================================
// Day 2: Block Tiling Kernel (structure only, still global loads)
// Each block computes one TILE x TILE output tile.
// ============================================================
__global__ void matmulBlockTiled(const float* __restrict__ A,
                                 const float* __restrict__ B,
                                 float* __restrict__ C,
                                 int M, int N, int K) {
    // TODO:
    // - compute tile origin (blockRow, blockCol)
    // - compute local (ty, tx) = threadIdx.y/x
    // - compute global (row, col)
    // - loop over k in [0, K) (still global reads)
    // - write output

    // Thread coordinates within the tile
    int blockRow = blockIdx.y * TILE;   // starting row of C tile
    int blockCol = blockIdx.x * TILE;   // starting col of C tile

    //Global coordinates (row, col) for the output element computed by this thread
    int row = blockRow + threadIdx.y;
    int col = blockCol + threadIdx.x;

    if (row >= M || col >= N)return;

    float acc = 0.0f;

    // K dimension tiling (still global loads)
    #pragma unroll
    for(int k_tile = 0; k_tile < K;  k_tile += TILE){
        for(int k_in_tile = 0; k_in_tile < TILE; k_in_tile++){
            int k = k_tile + k_in_tile;
            if(k < K){
                acc += A[row * K + k] * B[k * N + col];
            }
        }
    C[row * N + col] = acc;
    }
}

// ============================================================
// Day 3: Shared-Memory Tiled Kernel
// Load A/B tiles into shared memory, synchronize, compute partial sums.
// Also add a place to handle bank conflicts (e.g., padding) as TODO.
// ============================================================
__global__ void matmulSharedTiled(const float* __restrict__ A,
                                  const float* __restrict__ B,
                                  float* __restrict__ C,
                                  int M, int N, int K) {
    // TODO:
    // - declare shared memory tiles for A and B
    //   (optionally with padding for bank-conflict mitigation)
    // - compute (row, col)
    // - for each tile along K:
    //     * load A tile element into shared (guarded)
    //     * load B tile element into shared (guarded)
    //     * __syncthreads()
    //     * accumulate over tile dimension
    //     * __syncthreads()
    // - write C (guarded)

    // Thread coordinates within the tile
    int blockRow = blockIdx.y * TILE;   // starting row of C tile
    int blockCol = blockIdx.x * TILE;   // starting col of C tile

    //Global coordinates (row, col) for the output element computed by this thread
    int row = blockRow + threadIdx.y;
    int col = blockCol + threadIdx.x;

    if (row >= M || col >= N)return;

    extern __shared__ float As[TILE][TILE];
    extern __shared__ float Bs[TILE][TILE];

    float acc = 0.0f;

    // ---- K-tiling loop ----
    // t enumerates which K-tile we are processing
    for(int k_tile = 0; k_tile < K;  k_tile += TILE){
        //load A tile elements(row, k_tile + tx)
        int aRow = row;
        int aCol = k_tile + threadIdx.x;
        As[threadIdx.y][threadIdx.x] = (aRow < M && aCol < K)? A[aRow * K + aCol]:0.0f;

        //load B tile elements(k_tile + ty, col)

        int bRow = k_tile + threadIdx.y;
        int bCol = col;
        Bs[threadIdx.y][threadIdx.x] = (bRow < K && bCol < N)? A[bRow * N + bCol]:0.0f;

        __syncthreads();


        // Multiply-accumulate this K-tile
        #pragma unroll
        for(int kk = 0; kk < TILE; kk++){
            acc += As[threadIdx.y][kk] * Bs[kk][threadIdx.x];
        }
        __syncthreads();

    }
    if (row < M && col < N) {
        C[row * N + col] = acc;
        }
}

// ============================================================
// CPU reference GEMM (correctness baseline)
// ============================================================
static void matmulCPU(const float* A, const float* B, float* C, int M, int N, int K) {
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            double acc = 0.0;
            for (int k = 0; k < K; ++k) {
                acc += (double)A[i * K + k] * (double)B[k * N + j];
            }
            C[i * N + j] = (float)acc;
        }
    }
}

// ============================================================
// TODO: correctness checker
// Requirements:
//  - compare gpu vs ref within tol
//  - print first mismatch (i, gpu, ref)
// ============================================================
static bool checkClose(const float* gpu, const float* ref, int count, float tol) {
    // TODO
    for (int i = 0; i < count; ++i){
        if (fabsf(gpu[i] - ref[i]) >  tol){
            printf("mismatch in %d digit, cpu is %.2f, gpu is %.2f\n", i, ref[i], gpu[i]);

            return false;
        }
    }
    return true;
}

// ============================================================
// Timing helper (CUDA events) - implemented
// ============================================================
template <typename KernelFunc>
static float timeKernelMs(KernelFunc kernel,
                          dim3 grid, dim3 block,
                          const float* dA, const float* dB, float* dC,
                          int M, int N, int K,
                          int warmup, int iters) {
    for (int i = 0; i < warmup; ++i) {
        kernel<<<grid, block>>>(dA, dB, dC, M, N, K);
    }
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    CUDA_CHECK(cudaEventRecord(start));
    for (int i = 0; i < iters; ++i) {
        kernel<<<grid, block>>>(dA, dB, dC, M, N, K);
    }
    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));

    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));

    return ms / iters;
}

// ============================================================
// TODO (optional): GFLOPS estimator
// FLOPs ≈ 2*M*N*K
// ============================================================
static double estimateGFLOPS(int M, int N, int K, float ms) {
    // TODO
    double flops = 2.0 * (double)M * (double)N * (double)K;   // mul+add
    double gflops = flops / (ms * 1e-3) / 1e9;
    return gflops;
}

int main() {
    // Sizes (try non-multiples of TILE to test boundary guards)
    const int M = 1024;
    const int N = 1024;
    const int K = 1024;

    // Tolerance (float accumulation)
    const float tol = 1e-3f; // TODO (e.g., 1e-3 or 1e-2)

    const size_t bytesA = size_t(M) * K * sizeof(float);
    const size_t bytesB = size_t(K) * N * sizeof(float);
    const size_t bytesC = size_t(M) * N * sizeof(float);

    // Host alloc
    float* hA = (float*)std::malloc(bytesA);
    float* hB = (float*)std::malloc(bytesB);
    float* hC_ref = (float*)std::malloc(bytesC);
    float* hC_gpu = (float*)std::malloc(bytesC);

    if (!hA || !hB || !hC_ref || !hC_gpu) {
        fprintf(stderr, "Host malloc failed.\n");
        return EXIT_FAILURE;
    }

    // Init
    for (int i = 0; i < M * K; ++i) hA[i] = 0.001f * (i % 1000);
    for (int i = 0; i < K * N; ++i) hB[i] = 0.002f * (i % 1000);

    // CPU reference
    matmulCPU(hA, hB, hC_ref, M, N, K);

    // Device alloc
    float *dA=nullptr, *dB=nullptr, *dC=nullptr;
    CUDA_CHECK(cudaMalloc(&dA, bytesA));
    CUDA_CHECK(cudaMalloc(&dB, bytesB));
    CUDA_CHECK(cudaMalloc(&dC, bytesC));
    CUDA_CHECK(cudaMemcpy(dA, hA, bytesA, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(dB, hB, bytesB, cudaMemcpyHostToDevice));

    // ------------------------------------------------------------
    // TODO: choose block/grid for 2D kernels
    // Typical:
    //   block = (TILE, TILE)
    //   grid  = (ceil(N/TILE), ceil(M/TILE))
    // ------------------------------------------------------------

    dim3 block(TILE, TILE, 1); // TODO
    dim3 grid((N + TILE - 1)/TILE, (M + TILE - 1)/TILE, 1);  // TODO

    // ------------------------------------------------------------
    // TODO: Choose which kernel to run:
    //  - Day1: matmulNaive
    //  - Day2: matmulBlockTiled
    //  - Day3: matmulSharedTiled
    // ------------------------------------------------------------
    int which = 2; // TODO: 1/2/3

    // Launch for correctness + timing
    if (which == 1) {
        matmulNaive<<<grid, block>>>(dA, dB, dC, M, N, K);
    } else if (which == 2) {
        matmulBlockTiled<<<grid, block>>>(dA, dB, dC, M, N, K);
    } else {
        matmulSharedTiled<<<grid, block>>>(dA, dB, dC, M, N, K);
    }
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    CUDA_CHECK(cudaMemcpy(hC_gpu, dC, bytesC, cudaMemcpyDeviceToHost));

    // Correctness
    bool ok = checkClose(hC_gpu, hC_ref, M * N, tol);
    printf("Correctness: %s\n", ok ? "PASS" : "FAIL");

    // Timing
    const int warmup = 5;
    const int iters = 20;
    float ms = 0.0f;

    /*
    if (which == 1) {
        ms = timeKernelMs(matmulNaive, grid, block, dA, dB, dC, M, N, K, warmup, iters);
    } else if (which == 2) {
        ms = timeKernelMs(matmulBlockTiled, grid, block, dA, dB, dC, M, N, K, warmup, iters);
    } else {
        ms = timeKernelMs(matmulSharedTiled, grid, block, dA, dB, dC, M, N, K, warmup, iters);
    }*/



    ms = timeKernelMs(matmulNaive, grid, block, dA, dB, dC, M, N, K, warmup, iters);
    double gflops = estimateGFLOPS(M, N, K, ms);
    printf("[Naive MatMul Kernel (Baseline)]                          Kernel=%d | time=%.4f ms | GFLOPS=%.2f\n", 1, ms, gflops);


    ms = timeKernelMs(matmulBlockTiled, grid, block, dA, dB, dC, M, N, K, warmup, iters);
    gflops = estimateGFLOPS(M, N, K, ms);
    printf("[Block Tiling Kernel (structure only, still global loads)]Kernel=%d | time=%.4f ms | GFLOPS=%.2f\n", 2, ms, gflops);

    ms = timeKernelMs(matmulSharedTiled, grid, block, dA, dB, dC, M, N, K, warmup, iters);
    gflops = estimateGFLOPS(M, N, K, ms);
    printf("[Shared-Memory Tiled Kernel(fix size)]                    Kernel=%d | time=%.4f ms | GFLOPS=%.2f\n", 3, ms, gflops);


    ms = timeKernelMs(matmulSharedTiled, grid, block, dA, dB, dC, M, N, K, warmup, iters);
    gflops = estimateGFLOPS(M, N, K, ms);
    printf("[Shared-Memory Tiled Kernel(dynamic size)]                Kernel=%d | time=%.4f ms | GFLOPS=%.2f\n", 4, ms, gflops);

    // Cleanup
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    CUDA_CHECK(cudaFree(dC));
    std::free(hA);
    std::free(hB);
    std::free(hC_ref);
    std::free(hC_gpu);

    return ok ? EXIT_SUCCESS : EXIT_FAILURE;
}


Overwriting matmul_skeleton.cu


In [22]:
!nvcc -arch=sm_75 matmul_skeleton.cu -o matmul_skeleton
!./matmul_skeleton

matmul_skeleton.cu(122): warning #20044-D: extern declaration of the entity As is treated as a static definition

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

matmul_skeleton.cu(123): warning #20044-D: extern declaration of the entity Bs is treated as a static definition

Correctness: PASS
[Naive MatMul Kernel (Baseline)]                          Kernel=2 | time=9.1808 ms | GFLOPS=233.91
[Block Tiling Kernel (structure only, still global loads)]Kernel=2 | time=4.3786 ms | GFLOPS=490.45
[Shared-Memory Tiled Kernel(fix size)]                    Kernel=2 | time=2.6231 ms | GFLOPS=818.69
[Shared-Memory Tiled Kernel(dynamic size)]                Kernel=2 | time=2.4891 ms | GFLOPS=862.75
